In [1]:
import pandas as pd
import numpy as np
import models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Load MITBIH DATA

We took the first 186 values as input (instead of the first 187) because the PTBDB uses only 186 values as input

In [2]:
mitbih_train = pd.read_csv('exercise_data/heartbeat/mitbih_train.csv', header=None)

X = mitbih_train[list(range(186))].values[..., np.newaxis]
Y = mitbih_train[187]
X, Y = shuffle(X, Y, random_state=42)
Y_binary = to_categorical(Y)

# Train Model on MITBIH

In [3]:
rnn_model = models.LSTM_Model(outputs=5)
rnn_model.fit(X, Y_binary)
rnn_model.model.save('transfer_learning.h5')

Instructions for updating:
Colocations handled automatically by placer.
Train on 78798 samples, validate on 8756 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/1000
78720/78798 [============================>.] - ETA: 0s - loss: 0.3912 - acc: 0.8975
Epoch 00001: val_acc improved from -inf to 0.94164, saving model to LSTM_Model.h5
78798/78798 [==============================] - 24s 308us/sample - loss: 0.3912 - acc: 0.8975 - val_loss: 0.2244 - val_acc: 0.9416
Epoch 2/1000
78624/78798 [============================>.] - ETA: 0s - loss: 0.2043 - acc: 0.9450
Epoch 00002: val_acc improved from 0.94164 to 0.95683, saving model to LSTM_Model.h5
78798/78798 [==============================] - 23s 288us/sample - loss: 0.2043 - acc: 0.9450 - val_loss: 0.1596 - val_acc: 0.9568
Epoch 3/1000
78720/78798 [============================>.] - ETA: 0s - loss: 0.1600 - acc: 0.9544
Epoch 00003: val_acc improved from 0.95683 to 0.96071, saving model to LSTM_Model.h5
78798/78798 [===============

# LOAD PTBDB DATA


In [4]:
ptbdb_1 = pd.read_csv("exercise_data/heartbeat/ptbdb_normal.csv", header=None)
ptbdb_2 = pd.read_csv("exercise_data/heartbeat/ptbdb_abnormal.csv", header=None)
ptbdb = pd.concat([ptbdb_1, ptbdb_2])
X = ptbdb[list(range(186))].values[..., np.newaxis]
Y = ptbdb[187]
X, Y = shuffle(X, Y, random_state=42)

 train method for later use 

In [5]:
# train method for later use
def train(model, X, Y):
    early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
    redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=1)
    callbacks_list = [early, redonplat]  # early

    model.fit(X, Y, epochs=1000, verbose=1, callbacks=callbacks_list, validation_split=0.1)


# Freeze Everything expect the last layer

In [6]:
# Freeze everything except the last layer and train
model = load_model('transfer_learning.h5')
out = Dense(1, name='ouput')(model.layers[-2].output)
model2 = Model(model.input, out)

for x in range(4):
    model2.layers[x].trainable = False

model2.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['acc'])
model2.summary()
train(model2, X, Y)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 186, 1)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                8960      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
ouput (Dense)                (None, 1)                 65        
Total params: 17,345
Trainable params: 65
Non-trainable params: 17,280
_________________________________________________________________
Train on 13096 samples, validate on 1456 samples
Epoch 1/1000
13096/13096 [==============================] - 2s 178us/sample - loss: 1.82

# Don't freeze any layers

In [9]:
model = load_model('transfer_learning.h5')
out = Dense(1, name='ouput')(model.layers[-2].output)
model2 = Model(model.input, out)
model2.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['acc'])
model2.summary()
train(model2, X, Y)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 186, 1)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                8960      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
ouput (Dense)                (None, 1)                 65        
Total params: 17,345
Trainable params: 17,345
Non-trainable params: 0
_________________________________________________________________
Train on 13096 samples, validate on 1456 samples
Epoch 1/1000
13096/13096 [==============================] - 4s 338us/sample - loss: 0.972

# First Freeze - then unfreeze

## Freeze

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42, test_size=0.1)

model = load_model('transfer_learning.h5')
out = Dense(1, name='ouput')(model.layers[-2].output)
model2 = Model(model.input, out)

for x in range(4):
    model2.layers[x].trainable = False

model2.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['acc'])

early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=1)
callbacks_list = [early, redonplat]  # early

model2.summary()
model2.fit(X_train, Y_train, epochs=1000, verbose=1, callbacks=callbacks_list, validation_data=(X_test, Y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 186, 1)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                8960      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
ouput (Dense)                (None, 1)                 65        
Total params: 17,345
Trainable params: 65
Non-trainable params: 17,280
_________________________________________________________________
Train on 13096 samples, validate on 1456 samples
Epoch 1/1000
13096/13096 [==============================] - 3s 210us/sample - loss: 3.86

## Unfreeze

In [14]:
for x in range(4):
    model2.layers[x].trainable = True
    
model2.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['acc'])
model2.summary()
model2.fit(X_train, Y_train, epochs=1000, verbose=1, callbacks=callbacks_list, validation_data=(X_test, Y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 186, 1)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                8960      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
ouput (Dense)                (None, 1)                 65        
Total params: 17,345
Trainable params: 17,345
Non-trainable params: 0
_________________________________________________________________
Train on 13096 samples, validate on 1456 samples
Epoch 1/1000
13096/13096 [==============================] - 5s 363us/sample - loss: 0.546